In [23]:
s = 'ABCD'
s2 = '-asdfABCD'
s3 = '#'
s4 = 'abcd-'
s5 = 'ABD-'

In [106]:
import re
from icecream import ic

In [25]:
print(re.match('[A-Z]*[a-z]+|-[a-z][A-Z]*-*', s2))

<re.Match object; span=(0, 2), match='-a'>


In [26]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[A-Z]*[a-z]*|-[a-z][A-Z]*-*', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(0, 3), match='ABD'>


In [27]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('^[A-Z]+|^-|^[a-z]', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 3), match='ABD'>


In [28]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[^#]', elem))

<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='A'>


In [29]:
# comment for recommit

In [30]:
import numpy as np

In [143]:
def read_alignment(filename: str = 'test1.txt') -> list:
    '''
    DESCR
    -----
    Read in sequence alignment(s). 
    Entire file is line-by-line into a list object and stripped of whitespace.
    List will have representation of some row-wise separator in case of multiple alignments.
    
    ARGS
    -----
    filename: name of file
    
    RETURNS
    --------
    list of alignments separated by any separator (if applicable).
    '''
    with open(filename, 'r') as file:
        return [elem.strip() for elem in file.readlines()]
    
    '''
    with open(filename, 'r') as file:
        return [[*elem.strip()] for elem in file.readlines()]
    '''


ic(read_alignment());


def is_comparable():
    '''
    DESCR
    ------
    Checks to see if multiple alignments exist within an object 
    '''
    pass

alignments = read_alignment()

def columnarize(alignments: list) -> list:
    return [[*elem] for elem in alignments]

ic| read_alignment(): ['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT']


In [139]:
[[*elem] for elem in alignments]

[['A', 'A', 'A'],
 ['A', 'C', 'C'],
 ['A', 'C', 'G'],
 ['A', 'C', 'T'],
 ['#'],
 ['A', 'A', 'A', '-'],
 ['A', '-', 'C', 'C'],
 ['A', 'C', 'G', '-'],
 ['A', '-', 'C', 'T']]

In [109]:
nested_list = list()
temp_list = list()

for i in range(len(seqs)):
    if re.match('^[A-Z]+|^-|^[a-z]', seqs[i]):
        temp_list.append([*seqs[i]])
        ic(temp_list)
    else:
        nested_list.append(temp_list)
        ic(nested_list)
        temp_list = list()
        ic(temp_list)
        
nested_list.append(temp_list)

ic| seqs: ['AAA', 'ACC', 'ACG', 'ACT', '', 'AAA-', 'A-CC', 'ACG-', 'A-CT']
ic| temp_list: [['A', 'A', 'A']]
ic| temp_list: [['A', 'A', 'A'], ['A', 'C', 'C']]
ic| temp_list: [['A', 'A', 'A'], ['A', 'C', 'C'], ['A', 'C', 'G']]
ic| temp_list: [['A', 'A', 'A'], ['A', 'C', 'C'], ['A', 'C', 'G'], ['A', 'C', 'T']]
ic| nested_list: [[['A', 'A', 'A'], ['A', 'C', 'C'], ['A', 'C', 'G'], ['A', 'C', 'T']]]
ic| temp_list: []
ic| temp_list: [['A', 'A', 'A', '-']]
ic| temp_list: [['A', 'A', 'A', '-'], ['A', '-', 'C', 'C']]
ic| temp_list: [['A', 'A', 'A', '-'], ['A', '-', 'C', 'C'], ['A', 'C', 'G', '-']]
ic| temp_list: [['A', 'A', 'A', '-'],
                ['A', '-', 'C', 'C'],
                ['A', 'C', 'G', '-'],
                ['A', '-', 'C', 'T']]


In [91]:
range(len(nested_list[0]))

range(0, 4)

In [96]:
temp = ''
for i in range(len(nested_list[0])):
    print(nested_list[0][i][0])
    temp += nested_list[0][i][0]

A
A
A
A


In [97]:
temp

'AAAA'

In [101]:
from collections import Counter
c = Counter('asdf')
c

Counter({'a': 1, 's': 1, 'd': 1, 'f': 1})

In [104]:
Counter('AGC')

Counter({'A': 1, 'G': 1, 'C': 1})

In [ ]:
'AAA', 'ACC', 'ACG', 'ACT'

In [105]:
a = Counter('AAA')
print(a)
b = Counter('ACC')
print(b)
c = Counter('ACG')
print(c)
d = Counter('ACT')
print(d)

Counter({'A': 3})
Counter({'C': 2, 'A': 1})
Counter({'A': 1, 'C': 1, 'G': 1})
Counter({'A': 1, 'C': 1, 'T': 1})


In [81]:
np.array(nested_list, dtype = 'object').reshape(8,1)

array([list(['A', 'C', 'C'])], dtype=object)

In [58]:
np.array(nested_list, dtype = 'object').T

array([[list(['A', 'A', 'A']), list(['A', 'A', 'A', '-'])],
       [list(['A', 'C', 'C']), list(['A', '-', 'C', 'C'])],
       [list(['A', 'C', 'G']), list(['A', 'C', 'G', '-'])],
       [list(['A', 'C', 'T']), list(['A', '-', 'C', 'T'])]], dtype=object)

In [18]:
for i in range(len(seqs)):
    print(re.match('^[A-Z]+|^-|^[a-z]', seqs[i]))

<re.Match object; span=(0, 3), match='AAA'>
<re.Match object; span=(0, 3), match='ACC'>
<re.Match object; span=(0, 3), match='ACG'>
<re.Match object; span=(0, 3), match='ACT'>
None
<re.Match object; span=(0, 3), match='AAA'>
<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 3), match='ACG'>
<re.Match object; span=(0, 1), match='A'>


In [77]:
a = list(map(str.strip, a))
a

['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT']

array([['AAA', 'ACC', 'ACG', 'ACT']], dtype='<U3')

In [100]:
c[0][0]

'AAA'

In [30]:
for i in a:
    print(i if re.match('[^#]', i) else None)

AAA

ACC

ACG

ACT

None
AAA-

A-CC

ACG-

A-CT



In [32]:
with open('test1.txt', 'r') as file:
    a = np.array([elem.strip() for elem in file.readlines()], dtype = 'object')
    
a

array(['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT'],
      dtype=object)

In [37]:
def array_regex(i):
    return np.array([True if re.match('[^#]', i) else False])

ar = np.vectorize(array_regex)

c = ar(a)
c

array([ True,  True,  True,  True, False,  True,  True,  True,  True])

In [35]:
def array_regex(i):
    b = np.array([True if re.match('[^#]', i) else False])
    return np.where(True, b)

ar = np.vectorize(array_regex)

c = ar(a)

ValueError: either both or neither of x and y should be given

In [ ]:
print(a)
print(b)

array(['AAA\n', 'ACC\n', 'ACG\n', 'ACT\n', 'AAA-\n', 'A-CC\n', 'ACG-\n',
       'A-CT\n'], dtype=object)

In [45]:
b = np.array([])
for i in range(len(a)):
    print(i)

0
1
2
3
4
5
6
7
8


TypeError: slice indices must be integers or None or have an __index__ method